In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tabulate import tabulate

# Función estrategia 1 todas las cohortes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt

def preparado_estrategia1(datos, test_size=0.1, val_size=0.1111, random_seed=42):
    """
    Prepara el dataset para modelado dividiendo en conjuntos de entrenamiento 1, validación 2 y prueba 0,
    y añade una columna para indicar a qué conjunto pertenece cada fila. Además, imputa valores ausentes con la media para las columnas PC
    en función del cohorte al que pertenezcan
    y elimina filas con 'dem.time' NaN utilizando inplace=True para evitar la creación de copias adicionales habiendo demostrado que ninguna 
    de esas muestras convierten (llegan a ser casos) 
    También balancea las particiones de validation test y train en función de las clases y el género

    Parámetros:
    - datos: DataFrame de pandas.
    - test_size: Proporción del conjunto de prueba.
    - val_size: Proporción del conjunto de validación (del total de entrenamiento).
    - random_seed: Semilla para la generación de números aleatorios para reproducibilidad.

    Retorna:
    - DataFrame modificado con una nueva columna 'DataSet' con valor 1 para las muestras de train, 2 para validation y 3 para test.
    """
    # Comprobar que los dem.time nan son casos no convertidos
    muestras_requeridas = datos[(datos['dem.time'].isna()) & (datos['Status'] != 0)]
    numero_de_muestras = muestras_requeridas.shape[0]
    print(f'Número de muestras con dem.time NaN que hayan convertido: {numero_de_muestras}. Al no haber eliminamos esas muestras')
    
    # Eliminar filas donde 'dem.time' es NaN, modificación con inplace=True
    datos.dropna(subset=['dem.time'], inplace=True)
    
    # Separar la clase 'Status' directamente sin pasarla como parámetro
    y = datos['Status']
    # 'X' incluirá todas las columnas excepto la clase 'Status'
    X = datos.drop('Status', axis=1)
    
    # Crear una columna  que combine 'sex' con 'y' para usar en estratificación, quedando el df balanceado (dentro de lo posible) en función de sex y Status 
    stratify_labels = X['sex'].astype(str) + "_" + y.astype(str) # se pasa a string para evitar que sume números
    
    # División de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_seed, stratify=stratify_labels)
    
    # Necesitamos actualizar la etiqueta de estratificación para reflejar solo los datos de entrenamiento
    stratify_labels_train = X_train['sex'].astype(str) + "_" + y_train.astype(str)

    # División interna en el conjunto de entrenamiento para obtener un conjunto de validación
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=val_size, random_state=random_seed, stratify=stratify_labels_train)
    
     
    # Seleccionamos solo las columnas PC1, PC2, PC3, PC4
    pc_columns = ['PC1', 'PC2', 'PC3', 'PC4']

    # Calcula las medias por cohorte usando solo el conjunto de entrenamiento
    means_by_cohort_train = {pc: X_train.groupby('Cohort')[pc].mean() for pc in pc_columns}

    # Función que aplica la imputación basada en la media por cohorte
    def apply_imputation(dataset):
        for pc in pc_columns:
            for index, row in dataset.iterrows():
                if pd.isna(row[pc]):
                    cohort = row['Cohort']
                    if cohort in means_by_cohort_train[pc]:
                        dataset.at[index, pc] = means_by_cohort_train[pc][cohort]

    # Aplicar la imputación personalizada a cada uno de los conjuntos de datos
    apply_imputation(X_train)
    apply_imputation(X_val)
    apply_imputation(X_test)


    # Crear un array de etiquetas con el tamaño del DataFrame original, inicializado como 'train (1)'
    labels = np.array(['1'] * len(datos))

    # Configurar 'val' y 'test' según los índices de esos conjuntos
    labels[datos.index.isin(X_val.index)] = '2'
    labels[datos.index.isin(X_test.index)] = '3'

    # Añadir la columna de etiquetas al DataFrame original
    datos['DataSet'] = labels
    
    # Asegurar que 'Status' y 'sex' sean de tipo string para la visualización
    datos['Status'] = datos['Status'].astype(str)
    datos['sex'] = datos['sex'].astype(str)

    # Crear un conteo agregado para 'Status' y 'sex' por 'DataSet'
    conteo_agregado = datos.groupby(['DataSet', 'Status', 'sex']).size().reset_index(name='conteo')

    # Utilizar sns.catplot para crear una gráfica por cada valor de 'DataSet'
    g = sns.catplot(
        x='Status', 
        y='conteo', 
        hue='sex', 
        col='DataSet',
        data=conteo_agregado, 
        kind='bar',
        height=6, 
        aspect=1)

    # Añadir título y ajustes finales
    g.fig.subplots_adjust(top=0.9) # ajusta el título principal
    g.fig.suptitle('Distribución del paciente según su estado y género para las muestras de validación, entrenamiento y test')

    # Ajustar leyendas y etiquetas
    g.set_axis_labels('Estado clínico', 'Número de personas')

    # Mostrar las gráficas
    plt.show()

    # Devolver el DataFrame modificado
    return datos
    
    
 

# Función estrategia 2 para cohorte especifica

In [ ]:
def preparado_estrategia2(df, cohort_value, test_size=0.1, val_size=0.1111, random_seed=42):
    """
    Prepara el dataset filtrando por un valor específico en una columna de cohorte,
    divide los datos en conjuntos de entrenamiento, validación y prueba,
    y etiqueta cada muestra con '1'(train), '2'(validation) o '3' (test). Además, imputa valores ausentes con la media para las columnas PC
    y elimina filas con 'dem.time' NaN utilizando inplace=True para evitar la creación de copias adicionales habiendo demostrado que ninguna 
    de esas muestras convierten (llegan a ser casos) 
    También balancea las particiones de validation test y train en función de las clases y el género


    Parámetros:
    - df: DataFrame de pandas.
    - cohort_value: Valor del cohorte para filtrar las filas (puede ser string o numérico).
    - test_size: Proporción del tamaño del conjunto de prueba.
    - val_size: Proporción del tamaño del conjunto de validación del total de entrenamiento.
    - random_seed: Semilla para la generación de números aleatorios para reproducibilidad.

    Retorna:
    - DataFrame modificado con una nueva columna 'DataSet' que indica el conjunto asignado.
    """
    # Comprobar que los dem.time nan son casos no convertidos
    muestras_requeridas = df[(df['dem.time'].isna()) & (df['Status'] != 0)]
    numero_de_muestras = muestras_requeridas.shape[0]
    print(f'Número de muestras con dem.time NaN que hayan convertido: {numero_de_muestras}. Al no haber eliminamos esas muestras')
    
    # Eliminar filas donde 'dem.time' es NaN
    df.dropna(subset=['dem.time'], inplace=True)

    # Filtramos 'df' para incluir solo las filas donde la columna del cohorte tiene el valor especificado
    datos_filtered = df[df['Cohort'] == cohort_value].copy()
    print("Filas después de filtrar por Cohort:", datos_filtered.shape[0])


    # Separar la clase
    y = datos_filtered['Status']

    # 'X' incluirá todas las columnas excepto la columna de status
    X = datos_filtered.drop('Status', axis=1)

    # Crear una columna  que combine 'sex' con 'y' para usar en estratificación, quedando el df balanceado (dentro de lo posible) en función de sex y Status 
    stratify_labels = X['sex'].astype(str) + "_" + y.astype(str) # se pasa a string para evitar que sume números
    
    # División de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_seed, stratify=stratify_labels)
    
    # Necesitamos actualizar la etiqueta de estratificación para reflejar solo los datos de entrenamiento
    stratify_labels_train = X_train['sex'].astype(str) + "_" + y_train.astype(str)

    # División interna en el conjunto de entrenamiento para obtener un conjunto de validación
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=val_size, random_state=random_seed, stratify=stratify_labels_train)
    
    # Imputación de valores ausentes
    # Seleccionamos solo las columnas PC1, PC2, PC3, PC4
    pc_columns = ['PC1', 'PC2', 'PC3', 'PC4']

    # Creamos el imputador
    imputer = SimpleImputer(strategy='mean')

    # Ajustamos el imputador SOLO en las columnas seleccionadas del conjunto de entrenamiento
    imputer.fit(X_train[pc_columns])

    # Aplicamos la imputación a las mismas columnas en los conjuntos de entrenamiento, validación y prueba
    X_train[pc_columns] = imputer.transform(X_train[pc_columns])
    X_val[pc_columns] = imputer.transform(X_val[pc_columns])
    X_test[pc_columns] = imputer.transform(X_test[pc_columns])

    # Inicializar la columna 'DataSet' con '0' para todos
    datos_filtered['DataSet'] = '0'

    # Configurar 'train', 'val', y 'test' para los índices correctos en el DataFrame original
    datos_filtered.loc[datos_filtered.index.isin(X_train.index), 'DataSet'] = '1'
    datos_filtered.loc[datos_filtered.index.isin(X_val.index), 'DataSet'] = '2'
    datos_filtered.loc[datos_filtered.index.isin(X_test.index), 'DataSet'] = '3'
    
    # Transformar las columnas a tipo string para evitar problemas en la visualización
    datos_filtered['Status'] = datos_filtered['Status'].astype(str)
    datos_filtered['sex'] = datos_filtered['sex'].astype(str)
    
    # ojo esto no lo cambia en X ni en y

    # Crear un conteo agregado para 'Status' y 'sex' por 'DataSet'
    conteo_agregado = datos_filtered.groupby(['DataSet', 'Status', 'sex']).size().reset_index(name='conteo')

    # Crear un gráfico con sns.catplot
    g = sns.catplot(
        data=conteo_agregado,
        x='Status',
        y='conteo',
        hue='sex',
        col='DataSet',
        kind='bar',
        height=5,
        aspect=1
        )

    # Ajustar los títulos y etiquetas si es necesario
    g.fig.subplots_adjust(top=0.9) # ajusta el título principal
    g.set_axis_labels("Estado clínico", "Número de personas")
    g.fig.suptitle('Distribución del paciente según su estado y género para las muestras de validación, entrenamiento y test')

    # Mostrar gráfico
    plt.show()

    return datos_filtered

# Función estrategia 3 lista de varias cohortes

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

def preparado_estrategia3(df, cohort_values, test_size=0.1, val_size=0.1111, random_seed=42):
    """
    Prepares the dataset by filtering for specific values in a cohort column,
    splits the data into training, validation, and test sets,
    and labels each sample with '1' (train), '2' (validation), or '3' (test). Additionally, imputes missing values with the mean for PC columns
    and drops rows with 'dem.time' NaN using inplace=True to avoid creating additional copies, having shown that none of these samples convert (become cases).
    Also balances the validation, test, and train partitions based on classes and gender.

    Parameters:
    - df: pandas DataFrame.
    - cohort_values: List of cohort values to filter rows (can be string or numeric).
    - test_size: Proportion of the test set size.
    - val_size: Proportion of the validation set size from the total training set.
    - random_seed: Seed for random number generation for reproducibility.

    Returns:
    - Modified DataFrame with a new column 'DataSet' indicating the assigned set.
    """
    # Drop rows where 'dem.time' is NaN
    df.dropna(subset=['dem.time'], inplace=True)

    # Filter 'df' to include only rows where the cohort column has the specified values
    datos_filtered = df[df['Cohort'].isin(cohort_values)].copy()
    print("Rows after filtering by Cohort:", datos_filtered.shape[0])
    
    # Separate the class
    y = datos_filtered['Status']

    # 'X' will include all columns except the status column
    X = datos_filtered.drop('Status', axis=1)

    # Create a column that combines 'sex' with 'y' for stratification, making the df balanced (as much as possible) based on sex and Status
    stratify_labels = X['sex'].astype(str) + "_" + y.astype(str) # convert to string to avoid summing numbers
    
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_seed, stratify=stratify_labels)
    
    # Update the stratification label to reflect only the training data
    stratify_labels_train = X_train['sex'].astype(str) + "_" + y_train.astype(str)

    # Internal split in the training set to obtain a validation set
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=val_size, random_state=random_seed, stratify=stratify_labels_train)
    
    # Impute missing values
    # Select only PC1, PC2, PC3, PC4 columns
    pc_columns = ['PC1', 'PC2', 'PC3', 'PC4']

    # Create the imputer
    imputer = SimpleImputer(strategy='mean')

    # Fit the imputer ONLY on the selected columns of the training set
    imputer.fit(X_train[pc_columns])

    # Apply imputation to the same columns in the training, validation, and test sets
    X_train[pc_columns] = imputer.transform(X_train[pc_columns])
    X_val[pc_columns] = imputer.transform(X_val[pc_columns])
    X_test[pc_columns] = imputer.transform(X_test[pc_columns])

    # Initialize the 'DataSet' column with '0' for all
    datos_filtered['DataSet'] = '0'

    # Set 'train', 'val', and 'test' for the correct indices in the original DataFrame
    datos_filtered.loc[datos_filtered.index.isin(X_train.index), 'DataSet'] = '1'
    datos_filtered.loc[datos_filtered.index.isin(X_val.index), 'DataSet'] = '2'
    datos_filtered.loc[datos_filtered.index.isin(X_test.index), 'DataSet'] = '3'
    
    # Convert columns to string to avoid visualization issues
    #datos_filtered['Status'] = datos_filtered['Status'].astype(str)
    datos_filtered['sex'] = datos_filtered['sex'].astype(str)
    
    # This doesn't change X or y

    # Create an aggregated count for 'Status' and 'sex' by 'DataSet'
    aggregated_count = datos_filtered.groupby(['DataSet', 'Status', 'sex']).size().reset_index(name='count')

    # Create a plot with sns.catplot
    g = sns.catplot(
        data=aggregated_count,
        x='Status',
        y='count',
        hue='sex',
        col='DataSet',
        kind='bar',
        height=5,
        aspect=1
        )

    # Adjust titles and labels as needed
    g.fig.subplots_adjust(top=0.9) # adjust the main title
    g.set_axis_labels("Clinical Status", "Number of Patients")
    g.fig.suptitle('Patient Distribution by Clinical Status and Gender for Training, Validation, and Test Sets')

    # Show plot
    plt.show()

    return datos_filtered
